In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Phone Ranking Problem

Choosing a new phone is often quite a daunting task, with many different models from different companies and varying prices.

In [ ]:
data = pd.read_csv("./data/dataset.csv")
prefs = pd.read_csv("./data/preferences.csv")

In [ ]:
data

In [ ]:
crit_types = [True if crit.split("-")[1]=="gain" else False for crit in data.drop(columns="name").columns ]
crit_types


1. What is the domain of the problem about?
	
	The problem involves constinst of various phones and some of their parameters. 
	Price is given in złoty's, Ram in GB, Screen in inches, disk in GB, Oled and Foldable are binary variables and its beneficial for the phone to have them.
	OS ranks the operating systems with 0 being iOS, 1 being other, 2 being HarmonyOS and 3 being Android.

2. What is the source of the data?

	The data is taken from the internet from various online shops that offer these phones.

In [ ]:
def type_1(d):
	return 1 if d > 0 else 0

def type_5(d,q:float,p:float):
	if d > p:
		return 1
	elif d < q:
		return 0
	else: 
		return (d-q)/(p-q)

In [ ]:
def rank_string(ranking,names):
	_str = ""
	for i,n in sorted(zip(ranking,names),key=lambda a:a[0]):
		_str += f"{i}:\t{n}"
	return _str

def show_rank(ranking,names):
	print(rank_string(ranking,names))

In [ ]:
class Promethee:
	def __init__(self,data,criterion_type,criterion_weights=None,discrimination_thresholds=None):
		self.data = data.drop(columns="name")
		self.names = data["name"]
		self.ranking = [i for i in range(len(self.names))]
		self.is_crit_gain = criterion_type
		self.criterion_weighs = criterion_weights
		self.criterion_funcs = self._init_crit_funcs(discrimination_thresholds)

	def _init_crit_funcs(self,disc_thresh):
		return [lambda a,b: type5(a-b,q,p) if self.is_crit_gain else type5(b-a,q,p) for q,p in disc_thresh]
		

	def _cp_index(self,data:pd.DataFrame,criterion_weights:list[float]=None,criterion_func:list[callable]=None)->np.array:

		n = len(data)
		matrix = np.zeros((n,n))

		if criterion_weights==None:
			criterion_weights = [1 for _ in data.columns]

		if criterion_func == None:
			criterion_func = [type_1 for _ in criterion_weights]

		for c,criterion in enumerate(data.columns):
			for i,a in enumerate(data[criterion]):
				for j,b in enumerate(data[criterion][i+1:],i+1):
					pi = criterion_func[c]
					w = criterion_weights[c]
					matrix[i][j] += w * pi(a,b)
					matrix[j][i] += w * pi(b,a)
		
		return matrix/sum(criterion_weights)
	
	def _flow(cpi:np.array):
		positive_flow = np.sum(np.triu(cpi),axis=1)
		negative_flow = np.sum(np.triu(cpi.T),axis=1)

		return positive_flow,negative_flow
	
	def __str__(self):
		return rank_string(self.ranking,self.names)

In [ ]:
class Promethee1(Promethee):
	def rank(self):
		cpi = self._cp_index(data)
		pos_flow,neg_flow = self._flow(cpi)

		self.pos_rank = np.argsort(-pos_flow)
		self.neg_rank = np.argsort(neg_flow)

		

		return self.ranking

In [ ]:
class Promethee2(Promethee):
	def rank(self):
		cpi = self._cp_index(data)
		pos_flow,neg_flow = self._flow(cpi)

		f = pos_flow-neg_flow

		self.ranking = np.argsort(-f)

		return self.ranking

In [ ]:
estimated_weights = [7,5,3,4,4,8,5]

In [ ]:
cpi = cp_index(data.drop(columns="name"))
plt.imshow(cpi, interpolation='nearest')
plt.show()

In [ ]:
def 

In [ ]:
flow(cpi)

In [ ]:
show_rank(prom2_rank,data["name"])

In [ ]:
show_rank(prom1_pos,data["name"])

In [ ]:
show_rank(prom1_neg,data["name"])

In [ ]:
class ElectreTriB:
	def __init__(self,data:pd.DataFrame,criterion_type:list[bool],class_profiles:np.array, pref_indif_veto:np.array):
		"""
			class_profiles should be (criterion x profiles)

			pref_indif_veto should be(criterion x profiles x 3)
		"""
		self.data = data.drop(columns="name")
		self.names = data["name"]
		self.is_crit_gain = criterion_type
		self.class_profiles = class_profiles
		self.pref_indif_veto = pref_indif_veto

		self.b = len(self.class_profiles)
		self.g = len(self.data.columns)
		self.a = len(self.data)

	def _marginal_concordance(self):
		self.mconc = np.zeros((len(self.data),len(self.class_profiles)))
		for g_i,g in enumerate(self.data.columns):
			for a_i in range(self.a):
				for b_j in range(self.b):
					d = self.data[g][a_i] - self.class_profiles[g_i][b_j]
					if self.is_crit_gain[g_i]:
						d *= -1
					q,p = self.pref_indif_veto[g_i][b_j][0],self.pref_indif_veto[g_i][b_j][1]
					self.mconc = type_5(d,q,p)

	def _marginal_discordance(self):
		self.mdisc = np.zeros((len(self.data.columns),len(self.data),len(self.class_profiles)))
		for g_i,g in enumerate(self.data.columns):
			for a_i in range(self.a):
				for b_j in range(self.b):
					d = self.data[g][a_i] - self.class_profiles[g_i][b_j]
					if self.is_crit_gain[g_i]:
						d *= -1
					v,p = self.pref_indif_veto[g_i][b_j][2],self.pref_indif_veto[g_i][b_j][1]
					self.mdisc = type_5(d,v,p)